# How to use Python to create beautiful plots for scientific publications ?
# 2. Plot time series and highlight anomalies
### [matplotlib](https://matplotlib.org/stable/gallery/index.html)
### [seaborn](https://seaborn.pydata.org/examples/index.html)
### [pandas](https://pandas.pydata.org/pandas-docs/stable/user_guide/visualization.html#)

##### %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% 
# Import Packages

In [ ]:
import numpy as np 
import scipy.io as sio
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.gridspec as gridspec
from netCDF4 import Dataset
from mpl_toolkits.basemap import Basemap
from datetime import date

# Load data

In [ ]:
# Load grid information
grid_file = Dataset('/Users/z3533156/OneDrive - UNSW/Python/PUG/EACouter_varres_grd_mergedBLbry_uhroms.nc', 'r')
lon_rho   = grid_file.variables['lon_rho'][:,:]
lat_rho   = grid_file.variables['lat_rho'][:,:]
grid_file.close()
# Load MKE and EKE from AVISO
dataset    = sio.loadmat('/Users/z3533156/OneDrive - UNSW/Python/PUG/Figure1_data.mat')
lon_aviso  = np.transpose(np.tile(dataset['lon_aviso'],(1,102)))
lat_aviso  = np.tile(dataset['lat_aviso'],(1,69))
MKE_AVISO  = dataset['MKE_AVISO'][:,:]
EKE_AVISO  = dataset['EKE_AVISO'][:,:]
RMS_AVISO  = dataset['RMS_AVISO'][:,:]
UGOS_AVISO = dataset['UGOS_AVISO'][:,:]
VGOS_AVISO = dataset['VGOS_AVISO'][:,:]
AVISO_ts1  = dataset['EKE_AVISO_box_ts1'][:,0]      # 1-year low-pass time series
# Load MKE and EKE from ROMS
lon_roms   = dataset['lon_roms'][:,:]
lat_roms   = dataset['lat_roms'][:,:]
MKE_ROMS   = dataset['MKE_ROMS'][:,:]
EKE_ROMS   = dataset['EKE_ROMS'][:,:]
RMS_ROMS   = dataset['RMS_ROMS'][:,:]
UGOS_ROMS  = dataset['UGOS_ROMS'][:,:]
VGOS_ROMS  = dataset['VGOS_ROMS'][:,:]
# Load EKE time series
ROMS_ts_raw_450 = dataset['EKE_ROMS_box_ts_raw_450'][:,0] # original EKE450 time series
ROMS_ts1        = dataset['EKE_ROMS_box_ts1'][:,0]        # 1-year low-pass EKE0 time series
ROMS_ts_450     = dataset['EKE_ROMS_box_ts_450'][:,0]     # 1-year low-pass EKE450 time series
ROMS_ts_450     = ROMS_ts_450   + dataset['EKE_ROMS_box_mean_450'][0,0]
ROMS_mean_450   = ROMS_ts_450*0 + dataset['EKE_ROMS_box_mean_450'][0,0]
ROMS_pstd_450   = ROMS_mean_450 + dataset['EKE_ROMS_box_std_450'][0,0]
ROMS_nstd_450   = ROMS_mean_450 - dataset['EKE_ROMS_box_std_450'][0,0]

# Colors
### [List of named colors](https://matplotlib.org/stable/gallery/color/named_colors.html)
### [Specifying Colors](https://matplotlib.org/stable/tutorials/colors/colors.html)

In [ ]:
fig = plt.figure(figsize=(80, 80))
gs=gridspec.GridSpec(1,1)
# Generate time vector using datetime format (January 2 of year 1994 is day 1)
t = np.arange(date(1994,1,1).toordinal(),date(2016,9,29).toordinal()+1)
dates = [date.fromordinal(tt.astype(int)) for tt in t]
linefont = 12
fontscale=150
labelpadscale=20
padspacescale=50
#%% Panel 7
ax = fig.add_subplot(gs[0])
l1, b1, w1, h1 = ax.get_position().bounds
ax.set_position([l1-0.028, b1-0.01, w1*1.5, h1*0.8])
# Plot EKE time series
plt.plot(dates, ROMS_ts_raw_450, color='xkcd:medium grey', linewidth=linefont)
plt.plot(dates, AVISO_ts1, color='forestgreen', linewidth=linefont)
plt.plot(dates, ROMS_ts1, color='orange', linewidth=linefont)
plt.plot(dates, ROMS_ts_450,color='black', linewidth=linefont*2.0)
plt.plot(dates, ROMS_pstd_450,color='black', linewidth=linefont,linestyle='dashed')
plt.plot(dates, ROMS_mean_450,color='black', linewidth=linefont,linestyle='dashed')
plt.plot(dates, ROMS_nstd_450,color='black', linewidth=linefont,linestyle='dashed')
# Highlight the EKE anomalies
plt.fill_between(dates,ROMS_ts_450,ROMS_pstd_450, where = ROMS_ts_450>ROMS_pstd_450, color='red',zorder=3)
plt.fill_between(dates,ROMS_nstd_450,ROMS_ts_450, where = ROMS_nstd_450>ROMS_ts_450, color='dodgerblue',zorder=3)
# Add legend manually instead of using plt.legend
plt.plot([dates[1000],dates[1400]], [0.46,0.46], color='forestgreen', linewidth=linefont)
plt.text(dates[1500], 0.45, 'EKE(AVISO)',fontsize = fontscale*0.9, color='forestgreen')
plt.plot([dates[3200],dates[3600]], [0.46,0.46], color='orange', linewidth=linefont)
plt.text(dates[3700], 0.45, 'EKE(ROMS)',fontsize = fontscale*0.9, color='orange')
plt.plot([dates[5400],dates[5800]], [0.46,0.46], color='black', linewidth=linefont)
plt.text(dates[5900], 0.45, 'EKE450(ROMS)',fontsize = fontscale*0.9, color='black')
# Set x axis
plt.xlim(dates[0], dates[-1])
xminorLocator   = mpl.dates.YearLocator(1)
xmajorLocator   = mpl.dates.YearLocator(2)
xmajorFormatter = mpl.dates.DateFormatter('%Y')
ax.xaxis.set_minor_locator(xminorLocator)
ax.xaxis.set_major_locator(xmajorLocator)
ax.xaxis.set_major_formatter(xmajorFormatter)
plt.xticks(fontsize=fontscale*0.9)
ax.axes.xaxis.set_tick_params(pad=padspacescale)
plt.tick_params(axis='x',which='minor',bottom='on',top='on',length=50.0,width=20,colors='black',direction='in')
plt.xlabel('Year',fontsize=fontscale*0.9,labelpad=-10)
# ax.axes.xaxis.set_ticklabels([])  
# ax.axes.xaxis.set_ticklabels([])  
# Set y axis
plt.ylim(0,0.5)
ymajorLocator   = plt.MultipleLocator(0.1)
ymajorFormatter = plt.FormatStrFormatter('%2.1f')
ax.yaxis.set_major_locator(ymajorLocator)
ax.yaxis.set_major_formatter(ymajorFormatter)
plt.yticks(fontsize=fontscale*0.9)
ax.axes.yaxis.set_tick_params(pad=padspacescale)
plt.ylabel('EKE ($\mathregular{m^{2}/s^{2}}$)',fontsize=fontscale*0.9,labelpad=15)
# ax.axes.yaxis.set_ticklabels([])  
# Set length and width for the axis
ax.spines['bottom'].set_linewidth(labelpadscale)
ax.spines['left'].set_linewidth(labelpadscale)
ax.spines['top'].set_linewidth(labelpadscale)
ax.spines['right'].set_linewidth(labelpadscale)
plt.tick_params(bottom='on',left='on',top='on',right='on',length=80.0,width=20,colors='black',direction='in')

In [ ]:
fig = plt.figure(figsize=(80, 80))
gs=gridspec.GridSpec(1,1)
# Generate time vector using datetime format (January 2 of year 1994 is day 1)
t = np.arange(date(1994,1,1).toordinal(),date(2016,9,29).toordinal()+1)
dates = [date.fromordinal(tt.astype(int)) for tt in t]
linefont = 12
fontscale=150
labelpadscale=20
padspacescale=50
#%% Panel 7
ax = fig.add_subplot(gs[0])
l1, b1, w1, h1 = ax.get_position().bounds
ax.set_position([l1-0.028, b1-0.01, w1*1.5, h1*0.8])
# Plot EKE time series
plt.plot(dates, ROMS_ts_raw_450, color='xkcd:medium grey', linewidth=linefont)
plt.plot(dates, AVISO_ts1, color='forestgreen', linewidth=linefont)
plt.plot(dates, ROMS_ts1, color='orange', linewidth=linefont)
plt.plot(dates, ROMS_ts_450,color='black', linewidth=linefont*2.0)
plt.plot(dates, ROMS_pstd_450,color='black', linewidth=linefont,linestyle='dashed')
plt.plot(dates, ROMS_mean_450,color='black', linewidth=linefont,linestyle='dashed')
plt.plot(dates, ROMS_nstd_450,color='black', linewidth=linefont,linestyle='dashed')
# Highlight the EKE anomalies
plt.fill_between(dates,ROMS_ts_450,ROMS_pstd_450, where = ROMS_ts_450>ROMS_pstd_450, color='red',zorder=3)
plt.fill_between(dates,ROMS_nstd_450,ROMS_ts_450, where = ROMS_nstd_450>ROMS_ts_450, color='dodgerblue',zorder=3)
# Add legend manually instead of using plt.legend
plt.plot([dates[1000],dates[1400]], [0.46,0.46], color='forestgreen', linewidth=linefont)
plt.text(dates[1500], 0.45, 'EKE(AVISO)',fontsize = fontscale*0.9, color='forestgreen')
plt.plot([dates[3200],dates[3600]], [0.46,0.46], color='orange', linewidth=linefont)
plt.text(dates[3700], 0.45, 'EKE(ROMS)',fontsize = fontscale*0.9, color='orange')
plt.plot([dates[5400],dates[5800]], [0.46,0.46], color='black', linewidth=linefont)
plt.text(dates[5900], 0.45, 'EKE450(ROMS)',fontsize = fontscale*0.9, color='black')
# Set x axis
plt.xlim(dates[0], dates[-1])
xminorLocator   = mpl.dates.YearLocator(1)
xmajorLocator   = mpl.dates.YearLocator(2)
xmajorFormatter = mpl.dates.DateFormatter('%Y')
ax.xaxis.set_minor_locator(xminorLocator)
ax.xaxis.set_major_locator(xmajorLocator)
ax.xaxis.set_major_formatter(xmajorFormatter)
plt.xticks(fontsize=fontscale*0.9)
ax.axes.xaxis.set_tick_params(pad=padspacescale)
plt.tick_params(axis='x',which='minor',bottom='on',top='on',length=50.0,width=20,colors='black',direction='in')
plt.xlabel('Year',fontsize=fontscale*0.9,labelpad=-10)
# ax.axes.xaxis.set_ticklabels([])  
# Set y axis
plt.ylim(0,0.5)
ymajorLocator   = plt.MultipleLocator(0.1)
ymajorFormatter = plt.FormatStrFormatter('%2.1f')
ax.yaxis.set_major_locator(ymajorLocator)
ax.yaxis.set_major_formatter(ymajorFormatter)
plt.yticks(fontsize=fontscale*0.9)
ax.axes.yaxis.set_tick_params(pad=padspacescale)
plt.ylabel('EKE ($\mathregular{m^{2}/s^{2}}$)',fontsize=fontscale*0.9,labelpad=15)
# ax.axes.yaxis.set_ticklabels([])  
# Set length and width for the axis
ax.spines['bottom'].set_linewidth(labelpadscale)
ax.spines['left'].set_linewidth(labelpadscale)
ax.spines['top'].set_linewidth(labelpadscale)
ax.spines['right'].set_linewidth(labelpadscale)
plt.tick_params(bottom='on',left='on',top='on',right='on',length=80.0,width=20,colors='black',direction='in')

# Plot the distribution of AVISO MKE, EKE and RMS

In [ ]:
############################################################################
fig = plt.figure(figsize=(70, 95))
gs=gridspec.GridSpec(4,4)
# Creat the basemap for panel 1-3
m   = Basemap(projection='merc',llcrnrlat=np.nanmin(lat_roms),urcrnrlat= np.nanmax(lat_aviso),\
            llcrnrlon=np.nanmin(lon_roms),urcrnrlon=np.nanmax(lon_roms),resolution='h')
x, y = m(lon_aviso, lat_aviso)
# Locations of the Tasman EKE Box
box_lon=[114,184]
box_lat=[84,149]
lon = np.array([lon_roms[84,114],lon_roms[149,114],lon_roms[149,184],lon_roms[84,184],lon_roms[84,114]])
lat = np.array([lat_roms[84,114],lat_roms[149,114],lat_roms[149,184],lat_roms[84,184],lat_roms[84,114]])
x1,y1=m(lon,lat)
labelfont=145
padspacescale=50
labelpadscale=20
linefont = 12
scale  = 1.57
yy1 = np.arange(0, y.shape[0], 2)
xx1 = np.arange(0, x.shape[1], 2)
points1 = np.meshgrid(yy1, xx1)
labels1 = ['(a)MKE(AVISO)','(b)EKE(AVISO)','(c)RMS(AVISO)']
levels = np.linspace(0,0.32,100)
tick_marks = np.linspace(0,0.32,9)
cmaps1 = plt.cm.get_cmap('Spectral_r')
var_AVISO   = np.array([MKE_AVISO,EKE_AVISO,RMS_AVISO])
for i in range(3):
    ax = fig.add_subplot(gs[i])
    l, b, w, h = ax.get_position().bounds
    b=b-0.005
    if i==0:
        l=l-0.028
    elif i==1:
        l=l+0.042
    else:
        l=l+0.112
    m   = Basemap(projection='merc',llcrnrlat=np.nanmin(lat_roms),urcrnrlat= np.nanmax(lat_roms),\
            llcrnrlon=np.nanmin(lon_roms),urcrnrlon=np.nanmax(lon_roms),resolution='h')
    m.drawcoastlines(color='0.1',  linewidth=0.5*linefont)
    m.drawmapboundary(color='0.1', linewidth=0.5*linefont)
    m.fillcontinents(color='0.95', lake_color='white')
    m.drawstates(linewidth=8, linestyle='dashed', color='k', antialiased=1, ax=None, zorder=None)
    if i<1:
        m.drawparallels(np.arange(-45,-20, 2),labels=[1,0,0,0],linewidth=linefont,dashes=[2,2],color='.5',fontsize=labelfont*0.9)
    else:
        m.drawparallels(np.arange(-45,-20, 2),labels=[0,0,0,0],linewidth=linefont,dashes=[2,2],color='.5',fontsize=labelfont*0.9)
    m.drawmeridians(np.arange(145,165, 5),labels=[0,0,0,0],linewidth=linefont,dashes=[2,2],color='.5',fontsize=labelfont*0.9)
    # plot the boundary
    m.plot(x[0,:], y[0,:],linewidth=linefont, linestyle='solid', color='k')
    m.plot(x[:,0], y[:,0],linewidth=linefont, linestyle='solid', color='k')
    m.plot(x[-1,:], y[-1,:],linewidth=linefont, linestyle='solid', color='k')
    m.plot(x[:,-1], y[:,-1],linewidth=linefont, linestyle='solid', color='k')
    m.plot(x1,y1,linewidth=linefont,linestyle='solid', color='black')
    CB1=m.contourf(x, y, var_AVISO[i],cmap=cmaps1,levels=levels,origin='lower',extend='both')
    # Add quivers on MKE panel
    if i==0:
        Q1 = m.quiver(x[tuple(points1)],y[tuple(points1)],UGOS_AVISO[tuple(points1)],VGOS_AVISO[tuple(points1)],\
        color='black',width=0.005,scale=4.0)
        qk1 = plt.quiverkey(Q1, 0.18, 0.90, 0.5, '0.5m/s',labelpos='N',color='black',linewidth=linefont*1.0,fontproperties={'size': 0.9*labelfont})
    ax.spines['bottom'].set_linewidth(labelpadscale)
    ax.spines['left'].set_linewidth(labelpadscale)
    ax.spines['top'].set_linewidth(labelpadscale)
    ax.spines['right'].set_linewidth(labelpadscale)
    plt.title(labels1[i], fontsize=1.1*labelfont,loc='left',pad=padspacescale)
    ax.set_position([l, b, scale*w, scale*h])
    plt.tick_params(axis='both',which='major',bottom='on',left='on',length=50.0,width=20,colors='black',direction='out')
#     if i==2:
#         cx,cy = m(162.0, -24.4)
#         plt.text(cx, cy,'$\mathregular{m^{2}/s^{2}}$', color='black', fontsize=0.8*labelfont)
# Add the Australia map
ax = fig.add_subplot(gs[3])
l, b, w, h = ax.get_position().bounds
m   = Basemap(projection='merc',llcrnrlat=-50,urcrnrlat= -10.5,llcrnrlon=110,urcrnrlon=165,resolution='l')#,facecolor='0.95')
x, y = m(lon_aviso, lat_aviso)
m.drawcountries(color='forestgreen',  linewidth=1.0*linefont)
m.drawlsmask(land_color='0.95',ocean_color='red')
m.drawcoastlines(color='forestgreen',  linewidth=0.5*linefont)
m.plot(x[0,:], y[0,:],linewidth=linefont, linestyle='solid', color='forestgreen')
m.plot(x[:,0], y[:,0],linewidth=linefont, linestyle='solid', color='forestgreen')
m.plot(x[-1,:], y[-1,:],linewidth=linefont, linestyle='solid', color='forestgreen')
m.plot(x[:,-1], y[:,-1],linewidth=linefont, linestyle='solid', color='forestgreen')
m.drawmapboundary(color='1', linewidth=1.0*linefont)
m.fillcontinents(color='0.95', lake_color='white')
ax.set_position([l-0.62, b+0.15, 0.5*w, 0.5*h])
############################################################################

# Add the distribution of ROMS MKE, EKE and RMS

In [ ]:
############################################################################
fig = plt.figure(figsize=(70, 95))
gs=gridspec.GridSpec(4,4)
# Creat the basemap for panel 1-3
m   = Basemap(projection='merc',llcrnrlat=np.nanmin(lat_roms),urcrnrlat= np.nanmax(lat_aviso),\
            llcrnrlon=np.nanmin(lon_roms),urcrnrlon=np.nanmax(lon_roms),resolution='h')
x, y = m(lon_aviso, lat_aviso)
# Locations of the Tasman EKE Box
box_lon=[114,184]
box_lat=[84,149]
lon = np.array([lon_roms[84,114],lon_roms[149,114],lon_roms[149,184],lon_roms[84,184],lon_roms[84,114]])
lat = np.array([lat_roms[84,114],lat_roms[149,114],lat_roms[149,184],lat_roms[84,184],lat_roms[84,114]])
x1,y1=m(lon,lat)
labelfont=145
padspacescale=50
labelpadscale=20
linefont = 12
scale  = 1.57
yy1 = np.arange(0, y.shape[0], 2)
xx1 = np.arange(0, x.shape[1], 2)
points1 = np.meshgrid(yy1, xx1)
labels1 = ['(a)MKE(AVISO)','(b)EKE(AVISO)','(c)RMS(AVISO)']
levels = np.linspace(0,0.32,100)
tick_marks = np.linspace(0,0.32,9)
cmaps1 = plt.cm.get_cmap('Spectral_r')
var_AVISO   = np.array([MKE_AVISO,EKE_AVISO,RMS_AVISO])
for i in range(3):
    ax = fig.add_subplot(gs[i])
    l, b, w, h = ax.get_position().bounds
    b=b-0.005
    if i==0:
        l=l-0.028
    elif i==1:
        l=l+0.042
    else:
        l=l+0.112
    m   = Basemap(projection='merc',llcrnrlat=np.nanmin(lat_roms),urcrnrlat= np.nanmax(lat_roms),\
            llcrnrlon=np.nanmin(lon_roms),urcrnrlon=np.nanmax(lon_roms),resolution='h')
    m.drawcoastlines(color='0.1',  linewidth=0.5*linefont)
    m.drawmapboundary(color='0.1', linewidth=0.5*linefont)
    m.fillcontinents(color='0.95', lake_color='white')
    m.drawstates(linewidth=8, linestyle='dashed', color='k', antialiased=1, ax=None, zorder=None)
    if i<1:
        m.drawparallels(np.arange(-45,-20, 2),labels=[1,0,0,0],linewidth=linefont,dashes=[2,2],color='.5',fontsize=labelfont*0.9)
    else:
        m.drawparallels(np.arange(-45,-20, 2),labels=[0,0,0,0],linewidth=linefont,dashes=[2,2],color='.5',fontsize=labelfont*0.9)
    m.drawmeridians(np.arange(145,165, 5),labels=[0,0,0,0],linewidth=linefont,dashes=[2,2],color='.5',fontsize=labelfont*0.9)
    # plot the boundary
    m.plot(x[0,:], y[0,:],linewidth=linefont, linestyle='solid', color='k')
    m.plot(x[:,0], y[:,0],linewidth=linefont, linestyle='solid', color='k')
    m.plot(x[-1,:], y[-1,:],linewidth=linefont, linestyle='solid', color='k')
    m.plot(x[:,-1], y[:,-1],linewidth=linefont, linestyle='solid', color='k')
    m.plot(x1,y1,linewidth=linefont,linestyle='solid', color='black')
    CB1=m.contourf(x, y, var_AVISO[i],cmap=cmaps1,levels=levels,origin='lower',extend='both')
    # Add quivers on MKE panel
    if i==0:
        Q1 = m.quiver(x[tuple(points1)],y[tuple(points1)],UGOS_AVISO[tuple(points1)],VGOS_AVISO[tuple(points1)],\
        color='black',width=0.005,scale=4.0)
        qk1 = plt.quiverkey(Q1, 0.18, 0.90, 0.5, '0.5m/s',labelpos='N',color='black',linewidth=linefont*1.0,fontproperties={'size': 0.9*labelfont})
    ax.spines['bottom'].set_linewidth(labelpadscale)
    ax.spines['left'].set_linewidth(labelpadscale)
    ax.spines['top'].set_linewidth(labelpadscale)
    ax.spines['right'].set_linewidth(labelpadscale)
    plt.title(labels1[i], fontsize=1.1*labelfont,loc='left',pad=padspacescale)
    ax.set_position([l, b, scale*w, scale*h])
    plt.tick_params(axis='both',which='major',bottom='on',left='on',length=50.0,width=20,colors='black',direction='out')
    if i==2:
        cx,cy = m(162.0, -24.4)
        plt.text(cx, cy,'$\mathregular{m^{2}/s^{2}}$', color='black', fontsize=0.8*labelfont)
# Add the Australia map
ax = fig.add_subplot(gs[3])
l, b, w, h = ax.get_position().bounds
m   = Basemap(projection='merc',llcrnrlat=-50,urcrnrlat= -10.5,llcrnrlon=110,urcrnrlon=165,resolution='l')#,facecolor='0.95')
x, y = m(lon_aviso, lat_aviso)
m.drawcountries(color='forestgreen',  linewidth=1.0*linefont)
m.drawlsmask(land_color='0.95',ocean_color='red')
m.drawcoastlines(color='forestgreen',  linewidth=0.5*linefont)
m.plot(x[0,:], y[0,:],linewidth=linefont, linestyle='solid', color='forestgreen')
m.plot(x[:,0], y[:,0],linewidth=linefont, linestyle='solid', color='forestgreen')
m.plot(x[-1,:], y[-1,:],linewidth=linefont, linestyle='solid', color='forestgreen')
m.plot(x[:,-1], y[:,-1],linewidth=linefont, linestyle='solid', color='forestgreen')
m.drawmapboundary(color='1', linewidth=1.0*linefont)
m.fillcontinents(color='0.95', lake_color='white')
ax.set_position([l-0.62, b+0.15, 0.5*w, 0.5*h])
############################################################################
# Plot ROMS MEK, EKE and RMS
# Creat the basemap for panel 4-6
m   = Basemap(projection='merc',llcrnrlat=np.nanmin(lat_roms),urcrnrlat= np.nanmax(lat_roms),\
            llcrnrlon=np.nanmin(lon_roms),urcrnrlon=np.nanmax(lon_roms),resolution='h')
x, y = m(lon_roms, lat_roms)
yy2 = np.arange(10, y.shape[0], 15)
xx2 = np.arange(10, x.shape[1], 15)
points2 = np.meshgrid(yy2, xx2)
labels2 = ['(d)MKE(ROMS)','(e)EKE(ROMS)','(f)RMS(ROMS)']
var_ROMS = np.array([MKE_ROMS,EKE_ROMS,RMS_ROMS])
for i in range(3):
    ax = fig.add_subplot(gs[i+4])
    l, b, w, h = ax.get_position().bounds
    b=b-0.09
    if i==0:
        l=l-0.028
    elif i==1:
        l=l+0.042
    else:
        l=l+0.112
    m   = Basemap(projection='merc',llcrnrlat=np.nanmin(lat_roms),urcrnrlat= np.nanmax(lat_roms),\
            llcrnrlon=np.nanmin(lon_roms),urcrnrlon=np.nanmax(lon_roms),resolution='h')
    m.drawcoastlines(color='0.1',  linewidth=0.5*linefont)
    m.drawmapboundary(color='0.1', linewidth=0.5*linefont)
    m.fillcontinents(color='0.95', lake_color='white')
    m.drawstates(linewidth=8, linestyle='dashed', color='k', antialiased=1, ax=None, zorder=None)
    if i<1:
        m.drawparallels(np.arange(-45,-20, 2),labels=[1,0,0,0],linewidth=linefont*0.9,dashes=[2,2],color='.5',fontsize=labelfont*0.9)
    else:
        m.drawparallels(np.arange(-45,-20, 2),labels=[0,0,0,0],linewidth=linefont*0.9,dashes=[2,2],color='.5',fontsize=labelfont*0.9)
    m.drawmeridians(np.arange(145,165, 5),labels=[0,0,0,1],linewidth=linefont*0.9,dashes=[2,2],color='.5',fontsize=labelfont*0.9)
    # plot the boundary
    m.plot(x[0,:], y[0,:],linewidth=linefont, linestyle='solid', color='k')
    m.plot(x[:,0], y[:,0],linewidth=linefont, linestyle='solid', color='k')
    m.plot(x[-1,:], y[-1,:],linewidth=linefont, linestyle='solid', color='k')
    m.plot(x[:,-1], y[:,-1],linewidth=linefont, linestyle='solid', color='k')
    m.plot(x1,y1,linewidth=linefont,linestyle='solid', color='black')
    CB2=m.contourf(x, y, var_ROMS[i],cmap=cmaps1,levels=levels,origin='lower',extend='both')
    # Add quivers on MKE panel
    if i==0:
        Q2 = m.quiver(x[tuple(points2)],y[tuple(points2)],UGOS_ROMS[tuple(points2)],VGOS_ROMS[tuple(points2)],\
        color='black',width=0.005,scale=4.0)
        qk2 = plt.quiverkey(Q2, 0.18, 0.90, 0.5, '0.5m/s',labelpos='N',color='black',linewidth=linefont*1.0,fontproperties={'size': 0.9*labelfont})
    ax.spines['bottom'].set_linewidth(labelpadscale)
    ax.spines['left'].set_linewidth(labelpadscale)
    ax.spines['top'].set_linewidth(labelpadscale)
    ax.spines['right'].set_linewidth(labelpadscale)
    plt.title(labels2[i], fontsize=1.1*labelfont,loc='left',pad=padspacescale)
    ax.set_position([l, b, scale*w, scale*h])
    plt.tick_params(axis='both',which='major',bottom='on',left='on', length=50.0,width=20,colors='black',direction='out')
# Add colorbar
cbaxes1     = fig.add_axes([0.915, 0.42, 0.03, 0.56])
cb1 = plt.colorbar(CB1,orientation='vertical',cax = cbaxes1)
cb1.set_ticks(tick_marks)
cb1.ax.tick_params(labelsize=0.8*labelfont)
# Add the Australia map
ax = fig.add_subplot(gs[7])
l, b, w, h = ax.get_position().bounds
m   = Basemap(projection='merc',llcrnrlat=-50,urcrnrlat= -10.5,llcrnrlon=110,urcrnrlon=165,resolution='l')#,facecolor='0.95')
x, y = m(lon_roms, lat_roms)
m.drawcountries(color='forestgreen',  linewidth=1.0*linefont)
m.drawlsmask(land_color='0.95',ocean_color='red')
m.drawcoastlines(color='forestgreen',  linewidth=0.5*linefont)
m.plot(x[0,:], y[0,:],linewidth=linefont, linestyle='solid', color='forestgreen')
m.plot(x[:,0], y[:,0],linewidth=linefont, linestyle='solid', color='forestgreen')
m.plot(x[-1,:], y[-1,:],linewidth=linefont, linestyle='solid', color='forestgreen')
m.plot(x[:,-1], y[:,-1],linewidth=linefont, linestyle='solid', color='forestgreen')
m.drawmapboundary(color='1', linewidth=1.0*linefont)
m.fillcontinents(color='0.95', lake_color='white')
ax.set_position([l-0.62, b+0.065, 0.5*w, 0.5*h])

# Incorporate multipanels into one figure

In [ ]:
############################################################################
fig = plt.figure(figsize=(70, 95))
gs=gridspec.GridSpec(4,4)
# Creat the basemap for panel 1-3
m   = Basemap(projection='merc',llcrnrlat=np.nanmin(lat_roms),urcrnrlat= np.nanmax(lat_aviso),\
            llcrnrlon=np.nanmin(lon_roms),urcrnrlon=np.nanmax(lon_roms),resolution='h')
x, y = m(lon_aviso, lat_aviso)
# Locations of the Tasman EKE Box
box_lon=[114,184]
box_lat=[84,149]
lon = np.array([lon_roms[84,114],lon_roms[149,114],lon_roms[149,184],lon_roms[84,184],lon_roms[84,114]])
lat = np.array([lat_roms[84,114],lat_roms[149,114],lat_roms[149,184],lat_roms[84,184],lat_roms[84,114]])
x1,y1=m(lon,lat)
labelfont=145
padspacescale=50
labelpadscale=20
linefont = 12
scale  = 1.57
yy1 = np.arange(0, y.shape[0], 2)
xx1 = np.arange(0, x.shape[1], 2)
points1 = np.meshgrid(yy1, xx1)
labels1 = ['(a)MKE(AVISO)','(b)EKE(AVISO)','(c)RMS(AVISO)']
levels = np.linspace(0,0.32,100)
tick_marks = np.linspace(0,0.32,9)
cmaps1 = plt.cm.get_cmap('Spectral_r')
var_AVISO   = np.array([MKE_AVISO,EKE_AVISO,RMS_AVISO])
for i in range(3):
    ax = fig.add_subplot(gs[i])
    l, b, w, h = ax.get_position().bounds
    b=b-0.005
    if i==0:
        l=l-0.028
    elif i==1:
        l=l+0.042
    else:
        l=l+0.112
    m   = Basemap(projection='merc',llcrnrlat=np.nanmin(lat_roms),urcrnrlat= np.nanmax(lat_roms),\
            llcrnrlon=np.nanmin(lon_roms),urcrnrlon=np.nanmax(lon_roms),resolution='h')
    m.drawcoastlines(color='0.1',  linewidth=0.5*linefont)
    m.drawmapboundary(color='0.1', linewidth=0.5*linefont)
    m.fillcontinents(color='0.95', lake_color='white')
    m.drawstates(linewidth=8, linestyle='dashed', color='k', antialiased=1, ax=None, zorder=None)
    if i==0:
        m.drawparallels(np.arange(-45,-20, 2),labels=[1,0,0,0],linewidth=linefont,dashes=[2,2],color='.5',fontsize=labelfont*0.9)
    else:
        m.drawparallels(np.arange(-45,-20, 2),labels=[0,0,0,0],linewidth=linefont,dashes=[2,2],color='.5',fontsize=labelfont*0.9)
    m.drawmeridians(np.arange(145,165, 5),labels=[0,0,0,0],linewidth=linefont,dashes=[2,2],color='.5',fontsize=labelfont*0.9)
    # plot the boundary
    m.plot(x[0,:], y[0,:],linewidth=linefont, linestyle='solid', color='k')
    m.plot(x[:,0], y[:,0],linewidth=linefont, linestyle='solid', color='k')
    m.plot(x[-1,:], y[-1,:],linewidth=linefont, linestyle='solid', color='k')
    m.plot(x[:,-1], y[:,-1],linewidth=linefont, linestyle='solid', color='k')
    m.plot(x1,y1,linewidth=linefont,linestyle='solid', color='black')
    CB1=m.contourf(x, y, var_AVISO[i],cmap=cmaps1,levels=levels,origin='lower',extend='both')
    # Add quivers on MKE panel
    if i==0:
        Q1 = m.quiver(x[tuple(points1)],y[tuple(points1)],UGOS_AVISO[tuple(points1)],VGOS_AVISO[tuple(points1)],\
        color='black',width=0.005,scale=4.0)
        qk1 = plt.quiverkey(Q1, 0.18, 0.90, 0.5, '0.5m/s',labelpos='N',color='black',linewidth=linefont*1.0,fontproperties={'size': 0.9*labelfont})
    ax.spines['bottom'].set_linewidth(labelpadscale)
    ax.spines['left'].set_linewidth(labelpadscale)
    ax.spines['top'].set_linewidth(labelpadscale)
    ax.spines['right'].set_linewidth(labelpadscale)
    plt.title(labels1[i], fontsize=1.1*labelfont,loc='left',pad=padspacescale)
    ax.set_position([l, b, scale*w, scale*h])
    plt.tick_params(axis='both',which='major',bottom='on',left='on',length=50.0,width=20,colors='black',direction='out')
    if i==2:
        cx,cy = m(162.0, -24.4)
        plt.text(cx, cy,'$\mathregular{m^{2}/s^{2}}$', color='black', fontsize=0.8*labelfont)
# Add the Australia map
ax = fig.add_subplot(gs[3])
l, b, w, h = ax.get_position().bounds
m   = Basemap(projection='merc',llcrnrlat=-50,urcrnrlat= -10.5,llcrnrlon=110,urcrnrlon=165,resolution='l')#,facecolor='0.95')
x, y = m(lon_aviso, lat_aviso)
m.drawcountries(color='forestgreen',  linewidth=1.0*linefont)
m.drawlsmask(land_color='0.95',ocean_color='red')
m.drawcoastlines(color='forestgreen',  linewidth=0.5*linefont)
m.plot(x[0,:], y[0,:],linewidth=linefont, linestyle='solid', color='forestgreen')
m.plot(x[:,0], y[:,0],linewidth=linefont, linestyle='solid', color='forestgreen')
m.plot(x[-1,:], y[-1,:],linewidth=linefont, linestyle='solid', color='forestgreen')
m.plot(x[:,-1], y[:,-1],linewidth=linefont, linestyle='solid', color='forestgreen')
m.drawmapboundary(color='1', linewidth=1.0*linefont)
m.fillcontinents(color='0.95', lake_color='white')
ax.set_position([l-0.62, b+0.15, 0.5*w, 0.5*h])
############################################################################
# Plot ROMS MKE, EKE and RMS
# Creat the basemap for panel 4-6
m   = Basemap(projection='merc',llcrnrlat=np.nanmin(lat_roms),urcrnrlat= np.nanmax(lat_roms),\
            llcrnrlon=np.nanmin(lon_roms),urcrnrlon=np.nanmax(lon_roms),resolution='h')
x, y = m(lon_roms, lat_roms)
yy2 = np.arange(10, y.shape[0], 15)
xx2 = np.arange(10, x.shape[1], 15)
points2 = np.meshgrid(yy2, xx2)
labels2 = ['(d)MKE(ROMS)','(e)EKE(ROMS)','(f)RMS(ROMS)']
var_ROMS = np.array([MKE_ROMS,EKE_ROMS,RMS_ROMS])
for i in range(3):
    ax = fig.add_subplot(gs[i+4])
    l, b, w, h = ax.get_position().bounds
    b=b-0.09
    if i==0:
        l=l-0.028
    elif i==1:
        l=l+0.042
    else:
        l=l+0.112
    m   = Basemap(projection='merc',llcrnrlat=np.nanmin(lat_roms),urcrnrlat= np.nanmax(lat_roms),\
            llcrnrlon=np.nanmin(lon_roms),urcrnrlon=np.nanmax(lon_roms),resolution='h')
    m.drawcoastlines(color='0.1',  linewidth=0.5*linefont)
    m.drawmapboundary(color='0.1', linewidth=0.5*linefont)
    m.fillcontinents(color='0.95', lake_color='white')
    m.drawstates(linewidth=8, linestyle='dashed', color='k', antialiased=1, ax=None, zorder=None)
    if i<1:
        m.drawparallels(np.arange(-45,-20, 2),labels=[1,0,0,0],linewidth=linefont*0.9,dashes=[2,2],color='.5',fontsize=labelfont*0.9)
    else:
        m.drawparallels(np.arange(-45,-20, 2),labels=[0,0,0,0],linewidth=linefont*0.9,dashes=[2,2],color='.5',fontsize=labelfont*0.9)
    m.drawmeridians(np.arange(145,165, 5),labels=[0,0,0,1],linewidth=linefont*0.9,dashes=[2,2],color='.5',fontsize=labelfont*0.9)
    # plot the boundary
    m.plot(x[0,:], y[0,:],linewidth=linefont, linestyle='solid', color='k')
    m.plot(x[:,0], y[:,0],linewidth=linefont, linestyle='solid', color='k')
    m.plot(x[-1,:], y[-1,:],linewidth=linefont, linestyle='solid', color='k')
    m.plot(x[:,-1], y[:,-1],linewidth=linefont, linestyle='solid', color='k')
    m.plot(x1,y1,linewidth=linefont,linestyle='solid', color='black')
    CB2=m.contourf(x, y, var_ROMS[i],cmap=cmaps1,levels=levels,origin='lower',extend='both')
    # Add quivers on MKE panel
    if i==0:
        Q2 = m.quiver(x[tuple(points2)],y[tuple(points2)],UGOS_ROMS[tuple(points2)],VGOS_ROMS[tuple(points2)],\
        color='black',width=0.005,scale=4.0)
        qk2 = plt.quiverkey(Q2, 0.18, 0.90, 0.5, '0.5m/s',labelpos='N',color='black',linewidth=linefont*1.0,fontproperties={'size': 0.9*labelfont})
    ax.spines['bottom'].set_linewidth(labelpadscale)
    ax.spines['left'].set_linewidth(labelpadscale)
    ax.spines['top'].set_linewidth(labelpadscale)
    ax.spines['right'].set_linewidth(labelpadscale)
    plt.title(labels2[i], fontsize=1.1*labelfont,loc='left',pad=padspacescale)
    ax.set_position([l, b, scale*w, scale*h])
    plt.tick_params(axis='both',which='major',bottom='on',left='on', length=50.0,width=20,colors='black',direction='out')
# Add colorbar
cbaxes1     = fig.add_axes([0.915, 0.42, 0.03, 0.56])
cb1 = plt.colorbar(CB1,orientation='vertical',cax = cbaxes1)
cb1.set_ticks(tick_marks)
cb1.ax.tick_params(labelsize=0.8*labelfont)
# Add the Australia map
ax = fig.add_subplot(gs[7])
l, b, w, h = ax.get_position().bounds
m   = Basemap(projection='merc',llcrnrlat=-50,urcrnrlat= -10.5,llcrnrlon=110,urcrnrlon=165,resolution='l')#,facecolor='0.95')
x, y = m(lon_roms, lat_roms)
m.drawcountries(color='forestgreen',  linewidth=1.0*linefont)
m.drawlsmask(land_color='0.95',ocean_color='red')
m.drawcoastlines(color='forestgreen',  linewidth=0.5*linefont)
m.plot(x[0,:], y[0,:],linewidth=linefont, linestyle='solid', color='forestgreen')
m.plot(x[:,0], y[:,0],linewidth=linefont, linestyle='solid', color='forestgreen')
m.plot(x[-1,:], y[-1,:],linewidth=linefont, linestyle='solid', color='forestgreen')
m.plot(x[:,-1], y[:,-1],linewidth=linefont, linestyle='solid', color='forestgreen')
m.drawmapboundary(color='1', linewidth=1.0*linefont)
m.fillcontinents(color='0.95', lake_color='white')
ax.set_position([l-0.62, b+0.065, 0.5*w, 0.5*h])
############################################################################
# Generate time vector using datetime format (January 2 of year 1994 is day 1)
t = np.arange(date(1994,1,1).toordinal(),date(2016,9,29).toordinal()+1)
dates = [date.fromordinal(tt.astype(int)) for tt in t]
#%% Panel 7
ax = fig.add_subplot(gs[8:15])
l1, b1, w1, h1 = ax.get_position().bounds
ax.set_position([l1-0.028, b1-0.01, w1*1.5, h1*0.8])
# Plot EKE time series
plt.plot(dates, ROMS_ts_raw_450, color='darkgrey', linewidth=linefont)
plt.plot(dates, AVISO_ts1, color='forestgreen', linewidth=linefont)
plt.plot(dates, ROMS_ts1, color='orange', linewidth=linefont)
plt.plot(dates, ROMS_ts_450,color='black', linewidth=linefont*2.0)
plt.plot(dates, ROMS_pstd_450,color='black', linewidth=linefont,linestyle='dashed')
plt.plot(dates, ROMS_mean_450,color='black', linewidth=linefont,linestyle='dashed')
plt.plot(dates, ROMS_nstd_450,color='black', linewidth=linefont,linestyle='dashed')
# Highlight the EKE anomalies
plt.fill_between(dates,ROMS_ts_450,ROMS_pstd_450, where = ROMS_ts_450>ROMS_pstd_450, color='red',zorder=3)
plt.fill_between(dates,ROMS_nstd_450,ROMS_ts_450, where = ROMS_nstd_450>ROMS_ts_450, color='dodgerblue',zorder=3)
# Add legend manually instead of using plt.legend
plt.plot([dates[1000],dates[1400]], [0.46,0.46], color='forestgreen', linewidth=linefont)
plt.text(dates[1500], 0.45, 'EKE(AVISO)',fontsize = fontscale*0.9, color='forestgreen')
plt.plot([dates[3200],dates[3600]], [0.46,0.46], color='orange', linewidth=linefont)
plt.text(dates[3700], 0.45, 'EKE(ROMS)',fontsize = fontscale*0.9, color='orange')
plt.plot([dates[5400],dates[5800]], [0.46,0.46], color='black', linewidth=linefont)
plt.text(dates[5900], 0.45, 'EKE450(ROMS)',fontsize = fontscale*0.9, color='black')
# Set x axis
plt.xlim(dates[0], dates[-1])
xminorLocator   = mpl.dates.YearLocator(1)
xmajorLocator   = mpl.dates.YearLocator(2)
xmajorFormatter = mpl.dates.DateFormatter('%Y')
ax.xaxis.set_minor_locator(xminorLocator)
ax.xaxis.set_major_locator(xmajorLocator)
ax.xaxis.set_major_formatter(xmajorFormatter)
plt.xticks(fontsize=fontscale*0.9)
ax.axes.xaxis.set_tick_params(pad=padspacescale)
plt.tick_params(axis='x',which='minor',bottom='on',top='on',length=50.0,width=20,colors='black',direction='in')
plt.xlabel('Year',fontsize=fontscale*0.9,labelpad=-10)
# ax.axes.xaxis.set_ticklabels([])  
# Set y axis
plt.ylim(0,0.5)
ymajorLocator   = plt.MultipleLocator(0.1)
ymajorFormatter = plt.FormatStrFormatter('%2.1f')
ax.yaxis.set_major_locator(ymajorLocator)
ax.yaxis.set_major_formatter(ymajorFormatter)
plt.yticks(fontsize=fontscale*0.9)
ax.axes.yaxis.set_tick_params(pad=padspacescale)
plt.ylabel('EKE ($\mathregular{m^{2}/s^{2}}$)',fontsize=fontscale*0.9,labelpad=15)
# ax.axes.yaxis.set_ticklabels([])  
# Set length and width for the axis
ax.spines['bottom'].set_linewidth(labelpadscale)
ax.spines['left'].set_linewidth(labelpadscale)
ax.spines['top'].set_linewidth(labelpadscale)
ax.spines['right'].set_linewidth(labelpadscale)
plt.tick_params(bottom='on',left='on',top='on',right='on',length=80.0,width=20,colors='black',direction='in')